In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
from pprint import pformat

from hloc import extract_features, match_features, pairs_from_covisibility
from hloc import colmap_from_nvm, triangulation, localize_sfm, visualization
print("done")

done


# Pipeline for outdoor day-night visual localization

## Setup
Here we declare the paths to the dataset, the reconstruction and localization outputs, and we choose the feature extractor and the matcher. You only need to download the [Aachen Day-Night dataset](https://www.visuallocalization.net/datasets/) and put it in `datasets/aachen/`, or change the path.

In [15]:
dataset = Path('datasets/MMW_BLD/')  # change this if your dataset is somewhere else
images = dataset / 'images'

pairs = Path('pairs/cv/')
sfm_pairs = pairs / 'database_pairs-exhaustive.txt'  # top 20 most covisible in SIFT model
loc_pairs = pairs / 'pairs-query.txt'  # top 50 retrieved by NetVLAD

outputs = Path('outputs/CV/')  # where everything will be saved
reference_sfm = outputs / 'sfm_superpoint+superglue'  # the SfM model we will build
results = outputs / 'CV_hloc_superpoint+superglue_netvlad50.txt'  # the result file

In [3]:
# list the standard configurations available
print(f'Configs for feature extractors:\n{pformat(extract_features.confs)}')
print(f'Configs for feature matchers:\n{pformat(match_features.confs)}')

Configs for feature extractors:
{'d2net-ss': {'model': {'multiscale': False, 'name': 'd2net'},
              'output': 'feats-d2net-ss',
              'preprocessing': {'grayscale': False, 'resize_max': 1600}},
 'dir': {'model': {'name': 'dir'},
         'output': 'global-feats-dir',
         'preprocessing': {'resize_max': 1024}},
 'netvlad': {'model': {'name': 'netvlad'},
             'output': 'global-feats-netvlad',
             'preprocessing': {'resize_max': 1024}},
 'r2d2': {'model': {'max_keypoints': 5000, 'name': 'r2d2'},
          'output': 'feats-r2d2-n5000-r1024',
          'preprocessing': {'grayscale': False, 'resize_max': 1024}},
 'sift': {'model': {'name': 'sift'},
          'output': 'feats-sift',
          'preprocessing': {'grayscale': True, 'resize_max': 1600}},
 'superpoint_aachen': {'model': {'max_keypoints': 4096,
                                 'name': 'superpoint',
                                 'nms_radius': 3},
                       'output': 'feats-super

In [4]:
# pick one of the configurations for extraction and matching
# you can also simply write your own here!
feature_conf = extract_features.confs['superpoint_aachen']
matcher_conf = match_features.confs['superglue']

## Extract local features for database and query images

In [5]:
feature_path = extract_features.main(feature_conf, images, outputs)

[12/24/2021 08:57:36 INFO] Extracting local features with configuration:
{'model': {'max_keypoints': 4096, 'name': 'superpoint', 'nms_radius': 3},
 'output': 'feats-superpoint-n4096-r1024',
 'preprocessing': {'grayscale': True, 'resize_max': 1024}}
[12/24/2021 08:57:36 INFO] Found 81 images in root datasets/MMW_BLD/images.
Loaded SuperPoint model


100%|███████████████████████████████████████████| 81/81 [01:52<00:00,  1.39s/it]

[12/24/2021 08:59:29 INFO] Finished exporting features.


The function returns the path of the file in which all the extracted features are stored.

## Match the database images

In [12]:
sfm_match_path = match_features.main(matcher_conf, sfm_pairs, feature_conf['output'], outputs)

[12/24/2021 09:16:21 INFO] Matching local features with configuration:
{'model': {'name': 'superglue',
           'sinkhorn_iterations': 50,
           'weights': 'outdoor'},
 'output': 'matches-superglue'}
Loaded SuperGlue model ("outdoor" weights)


100%|█████████████████████████████████████| 1830/1830 [5:47:56<00:00, 11.41s/it]

[12/24/2021 15:04:19 INFO] Finished exporting matches.


The function returns the path of the file in which all the computed matches are stored.

## Triangulate a new SfM model from the given poses
We triangulate the sparse 3D pointcloud given the matches and the reference poses stored in the SIFT COLMAP model.

In [13]:
triangulation.main(
    reference_sfm,
    outputs / 'sfm_sift',
    images,
    sfm_pairs,
    feature_path,
    sfm_match_path,
    colmap_path='colmap')  # change if COLMAP is not in your PATH

[12/24/2021 20:40:34 INFO] Creating an empty model.
[12/24/2021 20:40:35 INFO] Importing features into the database...


100%|██████████████████████████████████████████| 61/61 [00:00<00:00, 325.30it/s]


[12/24/2021 20:40:35 INFO] Importing matches into the database...


100%|█████████████████████████████████████| 1830/1830 [00:00<00:00, 1950.12it/s]

[12/24/2021 20:40:36 INFO] Performing geometric verification of the matches...


[12/24/2021 20:41:59 INFO] Running the triangulation with command:
colmap point_triangulator --database_path outputs/CV/sfm_superpoint+superglue/database.db --image_path datasets/MMW_BLD/images --input_path outputs/CV/sfm_superpoint+superglue/empty --output_path outputs/CV/sfm_superpoint+superglue --Mapper.ba_refine_focal_length 0 --Mapper.ba_refine_principal_point 0 --Mapper.ba_refine_extra_params 0
[12/24/2021 20:42:04 INFO] Finished the triangulation with statistics:
{'mean_reproj_error': 1.34794,
 'mean_track_length': 3.670659,
 'num_observations': 79690,
 'num_observations_per_image': 1306.393443,
 'num_reg_images': 61,
 'num_sparse_points': 21710}


## Match the query images
Here we assume that the localization pairs are already computed using image retrieval (NetVLAD). To generate new pairs from your own global descriptors, have a look at `hloc/pairs_from_retrieval.py`. These pairs are also used for the localization - see below.

In [17]:
loc_match_path = match_features.main(matcher_conf, loc_pairs, feature_conf['output'], outputs)

[12/24/2021 20:46:08 INFO] Matching local features with configuration:
{'model': {'name': 'superglue',
           'sinkhorn_iterations': 50,
           'weights': 'outdoor'},
 'output': 'matches-superglue'}
Loaded SuperGlue model ("outdoor" weights)


100%|█████████████████████████████████████| 1220/1220 [4:00:22<00:00, 11.82s/it]

[12/25/2021 00:46:31 INFO] Finished exporting matches.


## Localize!
Perform hierarchical localization using the precomputed retrieval and matches. The file `Aachen_hloc_superpoint+superglue_netvlad50.txt` will contain the estimated query poses. Have a look at `Aachen_hloc_superpoint+superglue_netvlad50.txt_logs.pkl` to analyze some statistics and find failure cases.

In [19]:
localize_sfm.main(
    reference_sfm,
    dataset / 'queries/queries_with_intrinsics.txt',
    loc_pairs,
    feature_path,
    loc_match_path,
    results,
    covisibility_clustering=False)  # not required with SuperPoint+SuperGlue

[12/25/2021 01:22:01 INFO] Imported 20 images from queries_with_intrinsics.txt
[12/25/2021 01:22:01 INFO] Reading 3D model...
[12/25/2021 01:22:11 INFO] Starting localization...


100%|███████████████████████████████████████████| 20/20 [00:02<00:00,  7.30it/s]

[12/25/2021 01:22:14 INFO] Localized 20 / 20 images.
[12/25/2021 01:22:14 INFO] Writing poses to outputs/CV/CV_hloc_superpoint+superglue_netvlad50.txt...
[12/25/2021 01:22:14 INFO] Writing logs to outputs/CV/CV_hloc_superpoint+superglue_netvlad50.txt_logs.pkl...


[12/25/2021 01:22:14 INFO] Done!
